In [ ]:
import sqlite3 as sql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from bidi.algorithm import get_display
import arabic_reshaper
import tkinter as tk
from tkinter.filedialog import askopenfilename , asksaveasfile , askdirectory
import os


def open_file_water():
    
    global filepath_w
    filepath_w = askopenfilename (filetypes=[ ("All Files", "*.*")])
    if not filepath_w:
        return
    status_w.config(text = f"received - {filepath_w}")
    
def open_file_wastewater():
    
    global filepath_ww
    filepath_ww = askopenfilename (filetypes=[ ("All Files", "*.*")])
    if not filepath_ww:
        return
    status_ww.config(text = f"received - {filepath_ww}")
    
def write2sql():
    status_sql.config(text = "Writing to SQL")
    sql_address = asksaveasfile(defaultextension=".db",filetypes=[("SQL Database","*.db")])
    global conn
    conn = sql.connect(sql_address.name)
    df = pd.read_excel(filepath_w , dtype= 'str')
    df1 = pd.read_excel(filepath_ww , dtype= 'str')
    df.to_sql('Raw', conn, if_exists = 'replace', index= False)
    df1.to_sql('Raw_WW', conn, if_exists = 'replace', index= False)
    status_sql.config(text = f"Wrote to - {sql_address.name}")
    
def yesterday(date: str):
    dateList = date.split("/")
    lastMonthLength = 0
    
    if int(dateList[1]) > 7 and int(dateList[1]) <=12:
        lastMonthLength = 30
    elif int(dateList[1]) > 1 and int(dateList[1]) <=7:
        lastMonthLength = 31
    elif int(dateList[1]) == 1:
        lastMonthLength = 29
    
    y = {'year': dateList[0], 'month': dateList[1] , 'day': dateList[2]  }
    
    if int(dateList[2]) > 1:
        y['day'] = int(dateList[2]) - 1
        if y['day'] < 10:
            y['day'] = "0" + str(y['day'])
    elif int(dateList[2]) == 1 and int(dateList[1]) > 1:
        y['month'] = int(dateList[1]) - 1
        if y['month'] < 10:
            y['month'] = "0" + str(y['month'])
        if int(dateList[1]) <= 7:
            y['day'] = 31
        else:
            y['day'] = 30
    elif int(dateList[2]) == 1 and int(dateList[1]) == 1:
        y['year'] = int(dateList[0]) - 1
        y['month'] = 12
        y['day'] = 29
    
    return ("/".join(list(map(str, [y['year'], y['month'], y['day']]))))
    

def nDaysBefore(date: str , n: int):
    listOfDates = []
    for i in range(n):
        if i==0:
            listOfDates.append(yesterday(date_ent.get()))
        else:
            listOfDates.append(yesterday(listOfDates[-1]))
    return(list(reversed(listOfDates)))


def draw_charts():
    output_dir = askdirectory(initialdir = "c:\\")
    days = nDaysBefore(date_ent.get() , 30)
    df_all = pd.DataFrame(index= listOfProvinces , columns= days)
    for province in listOfProvinces:
        for day in days:
            querry = "SELECT DateRegister, TimeRegister, Company FROM RAW" + " WHERE Company = " + "\'" + province + "\'" + " AND DateRegister = " + "\'" + day + "\'"
            df = pd.read_sql( querry, conn, coerce_float= False)
            df_all.at[province, day] = df['DateRegister'].count()
        
    df_all_ww = pd.DataFrame(index= listOfProvinces , columns= days)
    for province in listOfProvinces:
        for day in days:
            querry = "SELECT DateRegister, TimeRegister, Company FROM RAW_WW" + " WHERE Company = " + "\'" + province + "\'" + " AND DateRegister = " + "\'" + day + "\'"
            df = pd.read_sql( querry, conn, coerce_float= False)
            df_all_ww.at[province, day] = df['DateRegister'].count()
    
    df_all.to_excel(output_dir + "\\Water.xlsx")
    df_all_ww.to_excel(output_dir + "\\Wastewater.xlsx" )
    standard_deviations = {}
    standard_deviations_ww = {}

    for ind in df_all.index:
        standard_deviations[ind] = np.std(df_all.loc[ind].to_numpy())

    for ind in df_all_ww.index:
        standard_deviations_ww[ind] = np.std(df_all_ww.loc[ind].to_numpy())
    
    querry = "SELECT DateRegister, TimeRegister, Company FROM RAW" + " WHERE DateRegister= " + "\'" + yesterday(date_ent.get()) + "\'" + " OR DateRegister = " + "\'" +  date_ent.get() + "\'"
    df = pd.read_sql( querry, conn, coerce_float= False)

    querry = "SELECT DateRegister, TimeRegister, Company FROM RAW_WW" + " WHERE DateRegister= " + "\'" + yesterday(date_ent.get()) + "\'" + " OR DateRegister = " + "\'" +  date_ent.get() + "\'"
    df_ww = pd.read_sql( querry, conn, coerce_float= False)

    df['Hour'] = ['0' for i in range(len(df['TimeRegister'])) ]
    for i in df.index:
        df.at[i , 'Hour'] = df.at[i, 'TimeRegister'][:2]


    df_ww['Hour'] = ['0' for i in range(len(df_ww['TimeRegister'])) ]
    for i in df_ww.index:
        df_ww.at[i , 'Hour'] = df_ww.at[i, 'TimeRegister'][:2]
    
    xl = [i for i in df_all.columns]
    xl1 = [i for i in range(len(xl))]

    for ind in df_all.index:
        yl = []
        for i in xl:
            yl.append(int(df_all[i][ind]))
        x = np.array(xl)
        y = np.array(yl)
        x1 = np.array(xl1)
        
        plt.figure(figsize=(18, 10 )) 
        plt.subplot(3, 2, 1)

        plt.xticks(rotation=90)
        plt.plot(x, y, linewidth=1.0)
    
        persian_title = arabic_reshaper.reshape(ind + " - آب")
        plot_title = get_display(persian_title)
        plt.title(plot_title)
        plt.xlabel("Date")
        plt.ylabel("Number of Incidents")
    
        slope, intercept, r, p, std_err = stats.linregress(x1, y)
        def myfunc(x):
            return slope * x + intercept

        mymodel = list(map(myfunc, x1))
        plt.scatter(x1, y)
        plt.plot(x1, mymodel , linestyle = 'dotted')
        
        plt.subplot(3, 2, 2)
        persian_title = arabic_reshaper.reshape("مقایسه ساعت به ساعت با روز قبل")
        plot_title = get_display(persian_title)
        plt.title(plot_title)
        plt.xlabel("Hour")
        plt.ylabel("Number of Incidents")

        X = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']
    
        ydValueCounts = df[(df['Company']== ind) & (df['DateRegister'] == yesterday(date_ent.get()))]['Hour'].value_counts()
        tdValueCounts = df[(df['Company']== ind) & (df['DateRegister'] == date_ent.get())]['Hour'].value_counts()
    
        Yyesterday = []
        for item in X:
            if item in ydValueCounts.index:
                Yyesterday.append(int(ydValueCounts[item]))
            else:
                Yyesterday.append(0)
        
        Ytoday = []
        for item in X:
            if item in tdValueCounts.index:
                Ytoday.append(int(tdValueCounts[item]))
            else:
                Ytoday.append(0)
                
        X_axis = np.arange(len(X))
        plt.bar(X_axis - 0.2, Yyesterday, 0.4, label = yesterday(date_ent.get()))
        plt.bar(X_axis + 0.2, Ytoday, 0.4, label = date_ent.get())
    
        plt.xticks(X_axis, X)
        plt.legend()
        
        
        yl_ww = []
        for i in xl:
            yl_ww.append(int(df_all_ww[i][ind]))
        x_ww = np.array(xl)
        y_ww = np.array(yl_ww)
        x_ww1 = np.array(xl1)
        
        plt.subplot(3, 2, 5)

        plt.xticks(rotation=90)
        plt.plot(x_ww, y_ww, linewidth=1.0 , c = '#4CAF50')
    
        persian_title = arabic_reshaper.reshape(ind + " - فاضلاب")
        plot_title = get_display(persian_title)
        plt.title(plot_title)
        plt.xlabel("Date")
        plt.ylabel("Number of Incidents")
    
        slope, intercept, r, p, std_err = stats.linregress(x_ww1, y_ww)
        def myfunc(x):
            return slope * x + intercept

        mymodel = list(map(myfunc, x_ww1))
        plt.scatter(x_ww1, y_ww , c = '#4CAF50')
        plt.plot(x_ww1, mymodel, linestyle = 'dotted' , c = 'hotpink')
        
        plt.subplot(3, 2, 6)
        persian_title = arabic_reshaper.reshape("مقایسه ساعت به ساعت با روز قبل")
        plot_title = get_display(persian_title)
        plt.title(plot_title)
        plt.xlabel("Hour")
        plt.ylabel("Number of Incidents")

   
        ydValueCounts = df_ww[(df_ww['Company']== ind) & (df_ww['DateRegister'] == yesterday(date_ent.get()))]['Hour'].value_counts()
        tdValueCounts = df_ww[(df_ww['Company']== ind) & (df_ww['DateRegister'] == date_ent.get())]['Hour'].value_counts()
    
        Yyesterday = []
        for item in X:
            if item in ydValueCounts.index:
                Yyesterday.append(int(ydValueCounts[item]))
            else:
                Yyesterday.append(0)
                
        Ytoday = []
        for item in X:
            if item in tdValueCounts.index:
                Ytoday.append(int(tdValueCounts[item]))
            else:
                Ytoday.append(0)
                
        X_axis = np.arange(len(X))
        plt.bar(X_axis - 0.2, Yyesterday, 0.4, label = yesterday(date_ent.get()))
        plt.bar(X_axis + 0.2, Ytoday, 0.4, label = date_ent.get())
    
        plt.xticks(X_axis, X)
        plt.legend()

        plt.savefig(output_dir + "//" + ind + "1.jpg" ,  dpi=600 , orientation='portrait')
        plt.close()





            
            

listOfProvinces = ['خراسان جنوبی', 'تهران', 'البرز', 'اصفهان', 'گیلان', 'سمنان',
       'چهار محال و بختیاری', 'کردستان', 'لرستان', 'زنجان',
       'آذربایجان شرقی', 'کرمان', 'یزد', 'هرمزگان', 'مرکزی',
       'آذربایجان غربی', 'خراسان شمالی', 'مشهد', 'خوزستان', 'همدان',
       'شیراز', 'سیستان و بلوچستان', 'کاشان', 'قم', 'مازندران', 'اهواز',
       'اردبیل', 'گلستان', 'ایلام', 'قزوین', 'کرمانشاه',
       'کهکیلویه و بویر احمد', 'فارس', 'بوشهر', 'سبزوار', 'خراسان رضوی']




window = tk.Tk()
window.title("Water and Wastewater Monthly Incidents")
window.rowconfigure(0, minsize=50, weight=1)
window.columnconfigure(1, minsize=200, weight=1)




status_w = tk.Label(text = "Please Enter")
status_ww = tk.Label(text = "Please Enter")
status_sql = tk.Label(text = "")
btn_open_w = tk.Button(window, text="Open Water Excel", command=open_file_water)
btn_open_ww = tk.Button(window, text="Open Wastewater Excel", command=open_file_wastewater)
btn_write = tk.Button(window, text="Write to SQL", command=write2sql)
date_ent = tk.StringVar()
lbl_getDate = tk.Label(window, text = 'Enter Date. Example: 1401/01/05')
day_field = tk.Entry(window, textvariable = date_ent )
btn_draw = tk.Button(window, text="Draw Charts", command=draw_charts)

btn_open_w.grid(row=0, column=0, sticky="ew", padx=5, pady=0)
btn_open_ww.grid(row=1, column=0, sticky="ew", padx=5, pady=0)
btn_write.grid(row=2, column=0, sticky="ew", padx=5, pady=15)

status_w.grid(row=0, column=1, sticky="ew" ,padx=5, pady=0)
status_ww.grid(row=1, column=1, sticky="ew" ,padx=5, pady=0)
status_sql.grid(row=2, column=1, sticky="ew" ,padx=5, pady=0)

lbl_getDate.grid(row=3, column=0, sticky="ew", padx=5, pady=5)
day_field.grid(row=3, column=1, sticky="ew", padx=5, pady=5)
btn_draw.grid(row=4, column=0, sticky="ew", padx=5, pady=15)

window.mainloop()